In [1]:
!pip install catboost
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 24.0 MB/s eta 0:00:00


In [2]:
import pandas as pd
import gc
from google.colab import drive
drive.mount('/content/drive')

data_root = '/content/drive/MyDrive/부트캠프/'
data_path = data_root + '11_파이널/'

Mounted at /content/drive


In [3]:
origin_train = pd.read_parquet(data_path + 'processed/train_all.parquet')
# origin_test = pd.read_parquet(data_path + 'processed/test_all.parquet')

In [4]:
segment = pd.read_csv(data_path + 'processed/segment.csv')
segment['E'] = (segment['Segment'].isin(['E'])).astype(int)

In [5]:
segment['E'].value_counts()

,count
E,
1,1922052
0,477948


In [6]:
x = origin_train.drop(['ID', '기준년월'], axis=1)
y = segment['E']

In [7]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
x_scaled = scaler.fit_transform(x)

In [8]:
from sklearn.model_selection import train_test_split

x_train1, x_val, y_train1, y_val = train_test_split(x_scaled, y, test_size=0.2, random_state=42, stratify=y)
x_train, x_opt, y_train, y_opt = train_test_split(x_train1, y_train1, test_size=0.2, random_state=42, stratify=y_train1)

In [9]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
x_sample, y_sample = smote.fit_resample(x_train, y_train)

In [10]:
import optuna
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score

def cat_objective(trial):
    params = {
        'depth': trial.suggest_int('depth', 5, 12),
        # 'class_weights': trial.suggest_float('class_weights', 1.0, 30.0),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'l2_leaf_reg': trial.suggest_int('l2_leaf_reg', 1, 10),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-3)
    }

    clf = CatBoostClassifier(depth= params.get('depth'),
                            #  class_weights= [1.0, params.get('class_weights')],
                             border_count= params.get('border_count'),
                             l2_leaf_reg= params.get('l2_leaf_reg'),
                             learning_rate = params.get('learning_rate'),
                             iterations=1000, early_stopping_rounds=20,
                             task_type="GPU", devices='0',
                             verbose=False,
                             random_state=42)
    clf.fit(x_sample, y_sample, eval_set=(x_opt, y_opt))

    y_val_pred = clf.predict(x_val)
    f1_macro = f1_score(y_val, y_val_pred, average='macro')

    return f1_macro

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(cat_objective, n_trials=10)

print("Best trial:")
trial = study.best_trial
print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2025-07-14 14:31:52,308] A new study created in memory with name: no-name-ef163287-d023-4173-90cd-fdcb0aff9128
/tmp/ipython-input-10-403305882.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-3)
[I 2025-07-14 14:42:08,763] Trial 0 finished with value: 0.8331279428919345 and parameters: {'depth': 11, 'border_count': 155, 'l2_leaf_reg': 7, 'learning_rate': 4.576466380880103e-05}. Best is trial 0 with value: 0.8331279428919345.
/tmp/ipython-input-10-403305882.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-3)
[I 2025-07-14 14:5

In [ ]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(depth=6,
                           learning_rate=5e-3,
                           early_stopping_rounds=20, iterations=100,
                           class_weights=[3, 1],
                           task_type="GPU", devices='0',
                           random_state=42)
model.fit(x_train, y_train, eval_set=(x_opt, y_opt))

0:	learn: 0.6867851	test: 0.6887603	best: 0.6887603 (0)	total: 144ms	remaining: 14.2s
1:	learn: 0.6804971	test: 0.6844322	best: 0.6844322 (1)	total: 265ms	remaining: 13s
2:	learn: 0.6744394	test: 0.6801976	best: 0.6801976 (2)	total: 359ms	remaining: 11.6s
3:	learn: 0.6684717	test: 0.6760291	best: 0.6760291 (3)	total: 444ms	remaining: 10.6s
4:	learn: 0.6624517	test: 0.6718896	best: 0.6718896 (4)	total: 538ms	remaining: 10.2s
5:	learn: 0.6566654	test: 0.6678483	best: 0.6678483 (5)	total: 626ms	remaining: 9.8s
6:	learn: 0.6509801	test: 0.6638750	best: 0.6638750 (6)	total: 711ms	remaining: 9.45s
7:	learn: 0.6456262	test: 0.6599390	best: 0.6599390 (7)	total: 806ms	remaining: 9.27s
8:	learn: 0.6399357	test: 0.6560433	best: 0.6560433 (8)	total: 889ms	remaining: 8.99s
9:	learn: 0.6344877	test: 0.6522416	best: 0.6522416 (9)	total: 971ms	remaining: 8.73s
10:	learn: 0.6289593	test: 0.6484589	best: 0.6484589 (10)	total: 1.05s	remaining: 8.52s
11:	learn: 0.6236708	test: 0.6447727	best: 0.6447727 (1

In [ ]:
from sklearn.metrics import f1_score

y_val_pred = model.predict(x_val)
f1_micro = f1_score(y_val, y_val_pred, average='micro')
print(f"F1 score (micro): {f1_micro:.4f}")

f1_macro = f1_score(y_val, y_val_pred, average='macro')
print(f"F1 score (macro): {f1_macro:.4f}")

F1 score (micro): 0.8791
F1 score (macro): 0.8277


In [ ]:
from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(y_val, y_val_pred)
print(conf_matrix)

[[ 79945  15645]
 [ 42402 342008]]


In [ ]:
from sklearn.metrics import roc_auc_score

y_val_pred_proba = model.predict_proba(x_val)[:, 1]
roc_auc = roc_auc_score(y_val, y_val_pred_proba)

In [ ]:
import pickle

with open(data_path + 'data/step1_SMOTE_CW31.dat', 'wb') as fp:
    pickle.dump(model, fp)
    pickle.dump(scaler, fp)